In [1]:
%config IPCompleter.use_jedi = False

## Emoji Predictor

### Get the Emoji Package

In [2]:
!pip install emoji

In [3]:
import emoji

In [4]:
# emoji.EMOJI_UNICODE

In [5]:
emoji_dictionary = {
    "0": "\u2764\uFE0F",
    "1": ":baseball:",
    "2": ":grinning_face_with_big_eyes:",
    "3": ":disappointed_face:",
    "4": ":fork_and_knife:",
    "5": ":hundred_points:",
    "6": ":fire:",
    "7": ":face_blowing_a_kiss:",
    "8": ":chestnut:",
    "9": ":flexed_biceps:"
}

In [6]:
for e in emoji_dictionary.values():
    print(emoji.emojize(e))

❤️
⚾
😃
😞
🍴
💯
🔥
😘
🌰
💪


### Processing a custom Dataset

In [7]:
import pandas as pd
import numpy as np

In [8]:
train = pd.read_csv("Dataset/train_emoji.csv",header=None)
test = pd.read_csv("Dataset/test_emoji.csv",header=None)

In [9]:
train.head()

,0,1,2,3
0,never talk to me again,3,NaN,NaN
1,I am proud of your achievements,2,NaN,NaN
2,It is the worst day in my life,3,NaN,NaN
3,Miss you so much,0,NaN,[0]
4,food is life,4,NaN,NaN


In [10]:
# Lets print the sentences with emojis
data = train.values
print(data.shape)

(132, 4)


In [11]:
X_train = train[0]
Y_train = train[1]

X_test = test[0]
Y_test = test[1]

In [12]:
for i in range(5):
    print(X_train[i],emoji.emojize(emoji_dictionary[str(Y_train[i])]))

never talk to me again 😞
I am proud of your achievements 😃
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴


### Convert Sentences to Embeddings
Using **Glove 6B.5d.txt**

In [13]:
f = open("Dataset/glove.6B.50d.txt",encoding="utf-8")

In [14]:
embeddings_index = {}
cnt = 0
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:],dtype='float')
    #print(word,coefs)
    embeddings_index[word]=coefs
f.close()

In [15]:
emb_dim = embeddings_index["eat"].shape[0]
print(emb_dim)

50


### Converting sentences into vectors (Embedding Layer Output)

In [16]:
def embedding_output(X):
    maxLen = 10
    embedding_out = np.zeros((X.shape[0],maxLen,emb_dim))
    
    for ix in range(X.shape[0]):
        X[ix] = X[ix].split()
        for ij in range(len(X[ix])):
            # go to every word in the current (ix) sentence
            try:
                embedding_out[ix][ij] = embeddings_index[X[ix][ij].lower()]
            except:
                embedding_out[ix][ij] = np.zeros((50,))
    return embedding_out

embeddings_matrix_train = embedding_output(X_train)
embeddings_matrix_test = embedding_output(X_test)

<ipython-input-16-76eaeaa16d4b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[ix] = X[ix].split()


In [17]:
print(X_train[0])
print(len(X_train[0]))

['never', 'talk', 'to', 'me', 'again']
5


In [18]:
print(embeddings_matrix_train.shape)
print(embeddings_matrix_test.shape)

(132, 10, 50)
(56, 10, 50)


In [19]:
Y_train.shape

(132,)

In [20]:
from keras.utils import to_categorical

In [21]:
Y_train = to_categorical(Y_train,num_classes=5)
Y_test = to_categorical(Y_test,num_classes=5)

In [22]:
print(Y_train.shape)
print(Y_test.shape)

(132, 5)
(56, 5)


### Define the RNN/LSTM model

In [23]:
from keras.models import Sequential
from keras.layers import *

In [24]:
model = Sequential()
model.add(LSTM(64,input_shape=(10,50)))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=["accuracy"])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 64)                29440     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 5)                 325       
_________________________________________________________________
activation (Activation)      (None, 5)                 0         
Total params: 29,765
Trainable params: 29,765
Non-trainable params: 0
_________________________________________________________________


In [25]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

# earlystop = EarlyStopping(monitor='val_accuracy',patience=10)
checkpoint = ModelCheckpoint("best_model.h5",monitor='val_loss',verbose=True,save_best_only=True)

hist = model.fit(embeddings_matrix_train, Y_train,
                 epochs=100,
                 callbacks=[checkpoint],
                 batch_size=64,
                 shuffle=True,
                 validation_split=0.2
                )

Epoch 1/100
1/2 [==============>...............] - ETA: 0s - loss: 1.6007 - accuracy: 0.2969
Epoch 00001: val_loss improved from inf to 1.60909, saving model to best_model.h5
2/2 [==============================] - 0s 243ms/step - loss: 1.5868 - accuracy: 0.3619 - val_loss: 1.6091 - val_accuracy: 0.2222
Epoch 2/100
1/2 [==============>...............] - ETA: 0s - loss: 1.6082 - accuracy: 0.2656
Epoch 00002: val_loss did not improve from 1.60909
2/2 [==============================] - 0s 11ms/step - loss: 1.5823 - accuracy: 0.2762 - val_loss: 1.6110 - val_accuracy: 0.2593
Epoch 3/100
1/2 [==============>...............] - ETA: 0s - loss: 1.5467 - accuracy: 0.4375
Epoch 00003: val_loss did not improve from 1.60909
2/2 [==============================] - 0s 12ms/step - loss: 1.5700 - accuracy: 0.3714 - val_loss: 1.6165 - val_accuracy: 0.2593
Epoch 4/100
1/2 [==============>...............] - ETA: 0s - loss: 1.5290 - accuracy: 0.4219
Epoch 00004: val_loss did not improve from 1.60909
2/2 [===

1/2 [==============>...............] - ETA: 0s - loss: 0.5675 - accuracy: 0.9062
Epoch 00029: val_loss improved from 1.13849 to 1.07589, saving model to best_model.h5
2/2 [==============================] - 0s 19ms/step - loss: 0.6898 - accuracy: 0.8190 - val_loss: 1.0759 - val_accuracy: 0.6296
Epoch 30/100
1/2 [==============>...............] - ETA: 0s - loss: 0.6293 - accuracy: 0.8281
Epoch 00030: val_loss improved from 1.07589 to 1.03792, saving model to best_model.h5
2/2 [==============================] - 0s 18ms/step - loss: 0.6344 - accuracy: 0.8190 - val_loss: 1.0379 - val_accuracy: 0.4815
Epoch 31/100
1/2 [==============>...............] - ETA: 0s - loss: 0.7305 - accuracy: 0.7188
Epoch 00031: val_loss improved from 1.03792 to 0.93686, saving model to best_model.h5
2/2 [==============================] - 0s 17ms/step - loss: 0.6506 - accuracy: 0.7905 - val_loss: 0.9369 - val_accuracy: 0.5926
Epoch 32/100
1/2 [==============>...............] - ETA: 0s - loss: 0.6378 - accuracy: 0.

Epoch 58/100
1/2 [==============>...............] - ETA: 0s - loss: 0.0506 - accuracy: 1.0000
Epoch 00058: val_loss did not improve from 0.81268
2/2 [==============================] - 0s 11ms/step - loss: 0.0722 - accuracy: 0.9905 - val_loss: 1.0245 - val_accuracy: 0.7778
Epoch 59/100
1/2 [==============>...............] - ETA: 0s - loss: 0.0681 - accuracy: 1.0000
Epoch 00059: val_loss did not improve from 0.81268
2/2 [==============================] - 0s 9ms/step - loss: 0.0931 - accuracy: 0.9905 - val_loss: 1.0156 - val_accuracy: 0.7778
Epoch 60/100
1/2 [==============>...............] - ETA: 0s - loss: 0.0938 - accuracy: 1.0000
Epoch 00060: val_loss did not improve from 0.81268
2/2 [==============================] - 0s 10ms/step - loss: 0.0901 - accuracy: 0.9905 - val_loss: 0.9864 - val_accuracy: 0.7037
Epoch 61/100
1/2 [==============>...............] - ETA: 0s - loss: 0.0959 - accuracy: 0.9844
Epoch 00061: val_loss did not improve from 0.81268
2/2 [==============================] 

1/2 [==============>...............] - ETA: 0s - loss: 0.0107 - accuracy: 1.0000
Epoch 00088: val_loss did not improve from 0.81268
2/2 [==============================] - 0s 10ms/step - loss: 0.0205 - accuracy: 1.0000 - val_loss: 1.2172 - val_accuracy: 0.7037
Epoch 89/100
1/2 [==============>...............] - ETA: 0s - loss: 0.0184 - accuracy: 1.0000
Epoch 00089: val_loss did not improve from 0.81268
2/2 [==============================] - 0s 10ms/step - loss: 0.0219 - accuracy: 1.0000 - val_loss: 1.1633 - val_accuracy: 0.7037
Epoch 90/100
1/2 [==============>...............] - ETA: 0s - loss: 0.0261 - accuracy: 1.0000
Epoch 00090: val_loss did not improve from 0.81268
2/2 [==============================] - 0s 9ms/step - loss: 0.0193 - accuracy: 1.0000 - val_loss: 1.1490 - val_accuracy: 0.7037
Epoch 91/100
1/2 [==============>...............] - ETA: 0s - loss: 0.0143 - accuracy: 1.0000
Epoch 00091: val_loss did not improve from 0.81268
2/2 [==============================] - 0s 10ms/ste

In [26]:
pred = model.predict_classes(embeddings_matrix_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [27]:
model.load_weights("best_model.h5")

In [28]:
model.evaluate(embeddings_matrix_test, Y_test)

2/2 [==============================] - 0s 997us/step - loss: 1.2062 - accuracy: 0.5714


[1.2062474489212036, 0.5714285969734192]

### Implementing Stacked LSTM

In [29]:
from keras.models import Sequential
from keras.layers import *

In [30]:
# We have not added a Embedding layer because it adds a lot of parameters to the model
# And for such a small dataset like this, it is better to use Transfer Learning Approach
model = Sequential()
model.add(LSTM(64,input_shape=(10,50), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64,return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=["accuracy"])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 10, 64)            29440     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 325       
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
Total params: 62,789
Trainable params: 62,789
Non-trainable params: 0
__________________________________________________

In [31]:
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("best_model2.h5",monitor='val_loss',verbose=True,save_best_only=True)

hist = model.fit(embeddings_matrix_train, Y_train,
                 epochs=150,
                 callbacks=[checkpoint],
                 batch_size=64,
                 shuffle=True,
                 validation_split=0.2
                )

Epoch 1/150
1/2 [==============>...............] - ETA: 0s - loss: 1.6418 - accuracy: 0.2500
Epoch 00001: val_loss improved from inf to 1.59725, saving model to best_model2.h5
2/2 [==============================] - 1s 353ms/step - loss: 1.6314 - accuracy: 0.2381 - val_loss: 1.5972 - val_accuracy: 0.1852
Epoch 2/150
1/2 [==============>...............] - ETA: 0s - loss: 1.5949 - accuracy: 0.2812
Epoch 00002: val_loss did not improve from 1.59725
2/2 [==============================] - 0s 15ms/step - loss: 1.5883 - accuracy: 0.2667 - val_loss: 1.5981 - val_accuracy: 0.2963
Epoch 3/150
1/2 [==============>...............] - ETA: 0s - loss: 1.5503 - accuracy: 0.3438
Epoch 00003: val_loss did not improve from 1.59725
2/2 [==============================] - 0s 16ms/step - loss: 1.5457 - accuracy: 0.3524 - val_loss: 1.6023 - val_accuracy: 0.2593
Epoch 4/150
1/2 [==============>...............] - ETA: 0s - loss: 1.5248 - accuracy: 0.3906
Epoch 00004: val_loss did not improve from 1.59725
2/2 [==

Epoch 29/150
1/2 [==============>...............] - ETA: 0s - loss: 0.5580 - accuracy: 0.8438
Epoch 00029: val_loss did not improve from 0.99186
2/2 [==============================] - 0s 12ms/step - loss: 0.4738 - accuracy: 0.8571 - val_loss: 1.0179 - val_accuracy: 0.6667
Epoch 30/150
1/2 [==============>...............] - ETA: 0s - loss: 0.4262 - accuracy: 0.8750
Epoch 00030: val_loss did not improve from 0.99186
2/2 [==============================] - 0s 14ms/step - loss: 0.3992 - accuracy: 0.8857 - val_loss: 1.0949 - val_accuracy: 0.6667
Epoch 31/150
1/2 [==============>...............] - ETA: 0s - loss: 0.3382 - accuracy: 0.9062
Epoch 00031: val_loss did not improve from 0.99186
2/2 [==============================] - 0s 14ms/step - loss: 0.3893 - accuracy: 0.9048 - val_loss: 1.0287 - val_accuracy: 0.6296
Epoch 32/150
1/2 [==============>...............] - ETA: 0s - loss: 0.3318 - accuracy: 0.9375
Epoch 00032: val_loss improved from 0.99186 to 0.94155, saving model to best_model2.h5


Epoch 59/150
1/2 [==============>...............] - ETA: 0s - loss: 0.0373 - accuracy: 1.0000
Epoch 00059: val_loss did not improve from 0.89997
2/2 [==============================] - 0s 12ms/step - loss: 0.0388 - accuracy: 1.0000 - val_loss: 1.2521 - val_accuracy: 0.7037
Epoch 60/150
1/2 [==============>...............] - ETA: 0s - loss: 0.0408 - accuracy: 1.0000
Epoch 00060: val_loss did not improve from 0.89997
2/2 [==============================] - 0s 13ms/step - loss: 0.0411 - accuracy: 1.0000 - val_loss: 1.1559 - val_accuracy: 0.7407
Epoch 61/150
1/2 [==============>...............] - ETA: 0s - loss: 0.0316 - accuracy: 1.0000
Epoch 00061: val_loss did not improve from 0.89997
2/2 [==============================] - 0s 12ms/step - loss: 0.0326 - accuracy: 1.0000 - val_loss: 1.2076 - val_accuracy: 0.6667
Epoch 62/150
1/2 [==============>...............] - ETA: 0s - loss: 0.0313 - accuracy: 1.0000
Epoch 00062: val_loss did not improve from 0.89997
2/2 [==============================]

1/2 [==============>...............] - ETA: 0s - loss: 0.0332 - accuracy: 1.0000
Epoch 00089: val_loss did not improve from 0.89997
2/2 [==============================] - 0s 11ms/step - loss: 0.0517 - accuracy: 0.9905 - val_loss: 1.5823 - val_accuracy: 0.5926
Epoch 90/150
1/2 [==============>...............] - ETA: 0s - loss: 0.0638 - accuracy: 0.9688
Epoch 00090: val_loss did not improve from 0.89997
2/2 [==============================] - 0s 11ms/step - loss: 0.0448 - accuracy: 0.9810 - val_loss: 1.5122 - val_accuracy: 0.7037
Epoch 91/150
1/2 [==============>...............] - ETA: 0s - loss: 0.0066 - accuracy: 1.0000
Epoch 00091: val_loss did not improve from 0.89997
2/2 [==============================] - 0s 12ms/step - loss: 0.0261 - accuracy: 0.9905 - val_loss: 1.5330 - val_accuracy: 0.6667
Epoch 92/150
1/2 [==============>...............] - ETA: 0s - loss: 0.0156 - accuracy: 1.0000
Epoch 00092: val_loss did not improve from 0.89997
2/2 [==============================] - 0s 12ms/st

Epoch 119/150
1/2 [==============>...............] - ETA: 0s - loss: 0.0255 - accuracy: 1.0000
Epoch 00119: val_loss did not improve from 0.89997
2/2 [==============================] - 0s 12ms/step - loss: 0.0296 - accuracy: 1.0000 - val_loss: 1.3469 - val_accuracy: 0.6667
Epoch 120/150
1/2 [==============>...............] - ETA: 0s - loss: 0.0165 - accuracy: 1.0000
Epoch 00120: val_loss did not improve from 0.89997
2/2 [==============================] - 0s 13ms/step - loss: 0.0166 - accuracy: 1.0000 - val_loss: 1.2758 - val_accuracy: 0.7037
Epoch 121/150
1/2 [==============>...............] - ETA: 0s - loss: 0.0103 - accuracy: 1.0000
Epoch 00121: val_loss did not improve from 0.89997
2/2 [==============================] - 0s 12ms/step - loss: 0.0098 - accuracy: 1.0000 - val_loss: 1.2109 - val_accuracy: 0.7037
Epoch 122/150
1/2 [==============>...............] - ETA: 0s - loss: 0.0160 - accuracy: 1.0000
Epoch 00122: val_loss did not improve from 0.89997
2/2 [===========================

Epoch 149/150
1/2 [==============>...............] - ETA: 0s - loss: 0.0048 - accuracy: 1.0000
Epoch 00149: val_loss did not improve from 0.89997
2/2 [==============================] - 0s 13ms/step - loss: 0.0050 - accuracy: 1.0000 - val_loss: 1.2810 - val_accuracy: 0.7407
Epoch 150/150
1/2 [==============>...............] - ETA: 0s - loss: 0.0098 - accuracy: 1.0000
Epoch 00150: val_loss did not improve from 0.89997
2/2 [==============================] - 0s 13ms/step - loss: 0.0084 - accuracy: 1.0000 - val_loss: 1.2806 - val_accuracy: 0.7407


In [32]:
# model.load_weights("best_model2.h5")

In [33]:
model.evaluate(embeddings_matrix_test, Y_test)

2/2 [==============================] - 0s 3ms/step - loss: 1.4346 - accuracy: 0.6250


[1.4345649480819702, 0.625]

### Visualising the results

In [34]:
pred = model.predict_classes(embeddings_matrix_test)

In [35]:
pred

array([4, 3, 2, 0, 2, 2, 1, 2, 4, 2, 1, 2, 0, 0, 1, 3, 2, 2, 3, 4, 0, 0,
       4, 0, 3, 1, 2, 0, 4, 2, 0, 1, 0, 2, 0, 1, 2, 4, 4, 2, 1, 0, 0, 1,
       2, 2, 2, 2, 3, 3, 1, 2, 3, 2, 3, 3], dtype=int64)

In [37]:
for i in range(30):
    print(' '.join(X_test[i]))
    print(emoji.emojize(emoji_dictionary[str(np.argmax(Y_test[i]))]))
    print(emoji.emojize(emoji_dictionary[str(pred[i])]))

I want to eat
🍴
🍴
he did not answer
😞
😞
he got a raise
😃
😃
she got me a present
❤️
❤️
ha ha ha it was so funny
😃
😃
he is a good friend
❤️
😃
I am upset
❤️
⚾
We had such a lovely dinner tonight
❤️
😃
where is the food
🍴
🍴
Stop making this joke ha ha ha
😃
😃
where is the ball
⚾
⚾
work is hard
😞
😃
This girl is messing with me
😞
❤️
are you serious ha ha
😃
❤️
Let us go play baseball
⚾
⚾
This stupid grader is not working
😞
😞
work is horrible
😞
😃
Congratulation for having a baby
😃
😃
stop messing around
😞
😞
any suggestions for dinner
🍴
🍴
I love taking breaks
❤️
❤️
you brighten my day
😃
❤️
I boiled rice
🍴
🍴
she is a bully
😞
❤️
Why are you feeling bad
😞
😞
I am upset
😞
⚾
I worked during my birthday
😞
😃
My grandmother is the love of my life
❤️
❤️
enjoy your break
😃
🍴
valentine day is near
❤️
😃
